In [1]:
using Revise

In [2]:
using JLD2
using FileIO
import LearningMPC

In [3]:
addprocs(10)

10-element Array{Int64,1}:
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11

In [ ]:
@everywhere using Flux
@everywhere using MLDataPattern
@everywhere using CoordinateTransformations
@everywhere using ProgressMeter
@everywhere import FluxExtensions

In [5]:
samples = load("../2018-02-07-hopper-smaller-grid/grid_search.jld2")["samples"];

In [6]:
filter!(samples) do sample
    sample.state[2] <= sample.state[1] && !any(isnan, sample.uJ)
end;
length(samples)

59931

In [7]:
features(sample::LearningMPC.Sample) = (sample.state, sample.uJ[:, 1], sample.uJ[:, 2:end])
data = features.(samples);
train_data, test_data = splitobs(shuffleobs(data), at=0.85);

In [8]:
@everywhere function attention_model()
    signals = Chain(Dense(4, 16))
    weights = Chain(
        LinearMap(UniformScaling(0.2)),
        Dense(4, 16, elu),
        Dense(16, 16, elu),
        softmax)
    model = FluxExtensions.Attention(signals, weights)
    @assert size(model(zeros(4))) == (1,)
            
    loss = (x, u, J) -> Flux.mse(model(x), u[2])
    model, loss
end

@everywhere function layered_model()
    model = Chain(
        LinearMap(UniformScaling(0.2)),
        Dense(4, 16, elu),
        Dense(16, 16, elu),
        Dense(16, 1))
    
    @assert size(model(zeros(4))) == (1,)
    loss = (x, u, J) -> Flux.mse(model(x), u[2])
    model, loss
end

@everywhere function attention_model_tangent(weight=0.05)
    signals = Chain(Dense(4, 16))
    weights = Chain(
        LinearMap(UniformScaling(0.2)),
        Dense(4, 16, elu),
        Dense(16, 16, elu),
        softmax,
    )
    model = FluxExtensions.TangentPropagator(FluxExtensions.Attention(signals, weights))
            
    @assert size.(model(zeros(4))) == ((1,), (1, 4))
    
    function loss(x, u, J)
        uhat, Jhat = model(x)
        (1 - weight) * Flux.mse(uhat, u[2]) + weight * Flux.mse(Jhat, J[2, :])
    end
    function test_loss(x, u, J)
        uhat, Jhat = model(x)
        Flux.mse(uhat, u[2])
    end
    model, loss, test_loss
end

@everywhere function layered_model_tangent(weight=0.05)
    model = FluxExtensions.TangentPropagator(Chain(
        LinearMap(UniformScaling(0.2)),
        Dense(4, 16, elu),
        Dense(16, 16, elu),
        Dense(16, 1)))
            
    @assert size.(model(zeros(4))) == ((1,), (1, 4))
    
    function loss(x, u, J)
        uhat, Jhat = model(x)
        (1 - weight) * Flux.mse(uhat, u[2]) + weight * Flux.mse(Jhat, J[2, :])
    end
    function test_loss(x, u, J)
        uhat, Jhat = model(x)
        Flux.mse(uhat, u[2])
    end
    model, loss, test_loss
end

In [ ]:
iterations = 200

attention_losses = @parallel (vcat) for (train, validation) in collect(kfolds(train_data; k=10))
    model, loss = attention_model()
    opt = Flux.ADADelta(params(model))
    (model, map(1:iterations) do i
        Flux.train!(loss, train, opt)
        (mean(xy -> Flux.Tracker.value(loss(xy...)), validation),)
    end)
end
@show attention_losses

layered_losses = @parallel (vcat) for (train, validation) in collect(kfolds(train_data; k=10))
    model, loss = layered_model()
    opt = Flux.ADADelta(params(model))
    (model, map(1:iterations) do i
        Flux.train!(loss, train, opt)
        (mean(xy -> Flux.Tracker.value(loss(xy...)), validation),)
    end)
end
@show layered_losses

attention_losses_tangent = @parallel (vcat) for (train, validation) in collect(kfolds(train_data; k=10))
    model, loss, test_loss = attention_model_tangent()
    opt = Flux.ADADelta(params(model))
    (model, map(1:iterations) do i
        Flux.train!(loss, train, opt)
        (mean(xy -> Flux.Tracker.value(loss(xy...)), validation), mean(xy -> Flux.Tracker.value(test_loss(xy...)), validation))
    end)
end
@show attention_losses_tangent

layered_losses_tangent = @parallel (vcat) for (train, validation) in collect(kfolds(train_data; k=10))
    model, loss, test_loss = layered_model_tangent()
    opt = Flux.ADADelta(params(model))
    (model, map(1:iterations) do i
        Flux.train!(loss, train, opt)
        (mean(xy -> Flux.Tracker.value(loss(xy...)), validation), mean(xy -> Flux.Tracker.value(test_loss(xy...)), validation))
    end)
end
@show layered_losses_tangent



In [18]:
save("losses2.jld2", "attention", attention_losses, "layered", layered_losses, "attention_tangent", attention_losses_tangent, "layered_tangent", layered_losses_tangent)

In [19]:
using Plots; gr()

Plots.GRBackend()

In [20]:
typeof(attention_losses[1])

Tuple{FluxExtensions.Attention{Flux.Chain,Flux.Chain},Array{Tuple{Float64},1}}

In [28]:
plt = plot(legend=nothing)
for (model, losses) in attention_losses
    plot!(plt, last.(losses), linecolor="red")
end
for (model, losses) in layered_losses
    plot!(plt, last.(losses), linecolor="blue")
end
for (model, losses) in attention_losses_tangent
    plot!(plt, last.(losses), linecolor="pink")
end
for (model, losses) in layered_losses_tangent
    plot!(plt, last.(losses), linecolor="skyblue")
end
plt

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 50 
 
 
 100 
 
 
 150 
 
 
 200 
 
 
 25 
 
 
 50 
 
 
 75 
 
 
 100 
 
 
 125 
 
 
 150 
 
 
 175 
 
<polyline clip-path="url(#clip0102)" style="stroke:#ff0000; stroke-width:1; stroke-opacity:1; fill:none" points="
 35.5169,26.6148 38.2546,66.4354 40.9923,130.944 43.73,180.765 46.4676,224.568 49.2053,259.118 51.943,285.567 54.6807,305.861 57.4183,321.318 60.156,332.667 
 62.8937,340.616 65.6314,346.009 68.3691,349.669 71.1067,352.234 73.8444,354.115 76.5821,355.548 79.3198,356.659 82.0575,357.511 84.7951,358.14 87.5328,358.573 
 90.2705,358.843 93.0082,358.985 95.7458,359.042 98.4835,359.048 101.221,359.029 103.959,359.002 106.697,358.974 109.434,358.95 112.172,358.931 114.91,358.917 
 117.647,358.906 120.385,358.898 123.123,358.892 125.86,358.884 128.598,358.876 131.336,358.865 134.073,358.852 136.811,358.834 139.549,358.812 142.286,358.786 
 145.024,358.755 147.762,358.72 150.499,358.681 153.237,358.645 155.975,358.628 158.712,358.632 161.45,358.644 164.188,358.647 166.925,358.662 169.663,358.701 
 172.401,358.772 175.139,358.918 177.876,359.32 180.614,360.252 183.352,360.937 186.089,361.256 188.827,361.488 191.565,361.715 194.302,361.943 197.04,362.161 
 199.778,362.338 202.515,362.495 205.253,362.646 207.991,362.795 210.728,362.942 213.466,363.085 216.204,363.225 218.941,363.36 221.679,363.488 224.417,363.608 
 227.154,363.719 229.892,363.821 232.63,363.914 235.367,363.998 238.105,364.075 240.843,364.146 243.58,364.215 246.318,364.282 249.056,364.353 251.794,364.43 
 254.531,364.514 257.269,364.608 260.007,364.714 262.744,364.84 265.482,364.994 268.22,365.181 270.957,365.401 273.695,365.656 276.433,365.912 279.17,366.126 
 281.908,366.267 284.646,366.329 287.383,366.338 290.121,366.323 292.859,366.304 295.596,366.291 298.334,366.283 301.072,366.275 303.809,366.266 306.547,366.257 
 309.285,366.25 312.022,366.242 314.76,366.235 317.498,366.23 320.235,366.232 322.973,366.244 325.711,366.267 328.449,366.3 331.186,366.34 333.924,366.386 
 336.662,366.434 339.399,366.482 342.137,366.527 344.875,366.57 347.612,366.61 350.35,366.649 353.088,366.687 355.825,366.728 358.563,366.774 361.301,366.825 
 364.038,366.883 366.776,366.949 369.514,367.021 372.251,367.096 374.989,367.172 377.727,367.248 380.464,367.322 383.202,367.397 385.94,367.472 388.677,367.547 
 391.415,367.622 394.153,367.697 396.89,367.772 399.628,367.846 402.366,367.917 405.104,367.986 407.841,368.053 410.579,368.116 413.317,368.177 416.054,368.235 
 418.792,368.29 421.53,368.342 424.267,368.39 427.005,368.434 429.743,368.473 432.48,368.507 435.218,368.536 437.956,368.559 440.693,368.575 443.431,368.583 
 446.169,368.583 448.906,368.574 451.644,368.558 454.382,368.536 457.119,368.508 459.857,368.474 462.595,368.435 465.332,368.392 468.07,368.344 470.808,368.292 
 473.545,368.235 476.283,368.173 479.021,368.107 481.759,368.037 484.496,367.962 487.234,367.883 489.972,367.801 492.709,367.715 495.447,367.627 498.185,367.536 
 500.922,367.444 503.66,367.35 506.398,367.255 509.135,367.16 511.873,367.065 514.611,366.97 517.348,366.877 520.086,366.785 522.824,366.695 525.561,366.609 
 528.299,366.526 531.037,366.446 533.774,366.371 536.512,366.3 539.25,366.232 541.987,366.168 544.725,366.106 547.463,366.048 550.2,365.993 552.938,365.942 
 555.676,365.895 558.414,365.851 561.151,365.81 563.889,365.771 566.627,365.734 569.364,365.696 572.102,365.659 574.84,365.62 577.577,365.579 580.315,365.537 
 
 "/>
<polyline clip-path="url(#clip0102)" style="stroke:#ff0000; stroke-width:1; stroke-opacity:1; fill:none" points="
 35.5169,46.9726 38.2546,84.908 40.9923,152.139 43.73,203.409 46.4676,242.622 49.2053,273.45 51.943,297.6 54.6807,316.383 57.4183,330.731 60.156,341.148 
 62.8937,348.321 65.6314,353.157 68.3691,356.438 71.1067,358.609 73.8444,359.928 76.5821,360.673 79.3198,361.047 82.0575,361.19 84.7951,361.211 87.5328,361.198 
 90.2705,361.211 93.0082,361

In [24]:
last.(last.(attention_losses))

10-element Array{Tuple{Float64},1}:
 (21.9033,)
 (19.6214,)
 (18.2502,)
 (21.8882,)
 (18.1163,)
 (19.6783,)
 (17.9095,)
 (19.0924,)
 (17.264,) 
 (20.5518,)

In [25]:
last.(last.(layered_losses))

10-element Array{Tuple{Float64},1}:
 (33.1317,)
 (21.0533,)
 (18.466,) 
 (23.562,) 
 (25.8672,)
 (23.8732,)
 (32.903,) 
 (19.8653,)
 (20.4046,)
 (20.7992,)

In [ ]:
layered_losses_tangent_0 = @parallel (vcat) for (train, validation) in collect(kfolds(train_data; k=10))
    model, loss, test_loss = layered_model_tangent(0.0)
    opt = Flux.ADADelta(params(model))
    (model, map(1:iterations) do i
        Flux.train!(loss, train, opt)
        (mean(xy -> Flux.Tracker.value(loss(xy...)), validation), mean(xy -> Flux.Tracker.value(test_loss(xy...)), validation))
    end)
end
@show layered_losses_tangent_0

In [32]:
plt = plot(legend=nothing)
for (model, losses) in layered_losses_tangent_0
    plot!(plt, last.(losses), linecolor="red")
end
for (model, losses) in layered_losses
    plot!(plt, last.(losses), linecolor="blue")
end
plt

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 50 
 
 
 100 
 
 
 150 
 
 
 200 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 50 
 
 
 60 
 
<polyline clip-path="url(#clip0502)" style="stroke:#ff0000; stroke-width:1; stroke-opacity:1; fill:none" points="
 28.9273,16.8903 31.6981,141.499 34.4689,229.798 37.2397,258.737 40.0105,278.436 42.7813,292.186 45.552,301.314 48.3228,305.933 51.0936,308.375 53.8644,309.895 
 56.6352,310.774 59.406,311.343 62.1768,311.762 64.9476,312.086 67.7184,312.657 70.4892,313.249 73.26,313.981 76.0308,314.875 78.8016,315.881 81.5723,316.791 
 84.3431,317.681 87.1139,318.555 89.8847,319.411 92.6555,320.18 95.4263,320.865 98.1971,321.564 100.968,322.315 103.739,323.131 106.509,324.031 109.28,325.022 
 112.051,326.109 114.822,327.287 117.593,328.551 120.363,329.893 123.134,331.297 125.905,332.734 128.676,334.163 131.447,335.583 134.217,336.997 136.988,338.353 
 139.759,339.56 142.53,340.513 145.301,341.258 148.071,341.896 150.842,342.451 153.613,342.946 156.384,343.439 159.155,343.941 161.925,344.368 164.696,344.657 
 167.467,344.906 170.238,345.231 173.008,345.612 175.779,346 178.55,346.399 181.321,346.782 184.092,347.137 186.862,347.467 189.633,347.809 192.404,348.198 
 195.175,348.62 197.946,349.028 200.716,349.399 203.487,349.73 206.258,350.038 209.029,350.334 211.8,350.62 214.57,350.893 217.341,351.153 220.112,351.399 
 222.883,351.63 225.654,351.847 228.424,352.047 231.195,352.231 233.966,352.398 236.737,352.548 239.508,352.684 242.278,352.799 245.049,352.892 247.82,352.963 
 250.591,353.015 253.361,353.054 256.132,353.079 258.903,353.09 261.674,353.09 264.445,353.079 267.215,353.053 269.986,353.01 272.757,352.989 275.528,352.996 
 278.299,353.027 281.069,353.081 283.84,353.16 286.611,353.268 289.382,353.406 292.153,353.574 294.923,353.763 297.694,353.965 300.465,354.169 303.236,354.352 
 306.007,354.503 308.777,354.615 311.548,354.684 314.319,354.713 317.09,354.711 319.86,354.68 322.631,354.631 325.402,354.571 328.173,354.507 330.944,354.439 
 333.714,354.372 336.485,354.309 339.256,354.254 342.027,354.21 344.798,354.176 347.568,354.154 350.339,354.145 353.11,354.149 355.881,354.17 358.652,354.206 
 361.422,354.256 364.193,354.303 366.964,354.324 369.735,354.362 372.506,354.437 375.276,354.536 378.047,354.649 380.818,354.763 383.589,354.871 386.359,354.969 
 389.13,355.056 391.901,355.127 394.672,355.183 397.443,355.228 400.213,355.261 402.984,355.281 405.755,355.293 408.526,355.291 411.297,355.281 414.067,355.251 
 416.838,355.194 419.609,355.115 422.38,355.036 425.151,354.97 427.921,354.942 430.692,354.968 433.463,355.018 436.234,355.028 439.005,354.982 441.775,354.866 
 444.546,354.697 447.317,354.468 450.088,354.195 452.859,353.893 455.629,353.579 458.4,353.275 461.171,353.003 463.942,352.775 466.712,352.59 469.483,352.45 
 472.254,352.354 475.025,352.295 477.796,352.271 480.566,352.275 483.337,352.298 486.108,352.335 488.879,352.375 491.65,352.419 494.42,352.469 497.191,352.538 
 499.962,352.618 502.733,352.661 505.504,352.688 508.274,352.757 511.045,352.849 513.816,352.927 516.587,352.987 519.358,353.031 522.128,353.064 524.899,353.087 
 527.67,353.1 530.441,353.109 533.211,353.114 535.982,353.097 538.753,353.052 541.524,352.983 544.295,352.884 547.065,352.757 549.836,352.611 552.607,352.5 
 555.378,352.465 558.149,352.457 560.919,352.419 563.69,352.368 566.461,352.306 569.232,352.227 572.003,352.128 574.773,352.013 577.544,351.886 580.315,351.752 
 
 "/>
<polyline clip-path="url(#clip0502)" style="stroke:#ff0000; stroke-width:1; stroke-opacity:1; fill:none" points="
 28.9273,34.2593 31.6981,192.044 34.4689,248.02 37.2397,272.433 40.0105,289.804 42.7813,300.198 45.552,305.604 48.3228,308.14 51.0936,309.296 53.8644,309.784 
 56.6352,309.897 59.406,309.778 62.1768,309.745 64.9476,309.881 67.7184,310.084 70.4892,310.315 73.26,310.609 76.0308,310.804 78.8016,310.821 81.5723,310.703 
 84.3431,310.624 87.1139,310.683 89.8847,310.847 92.655

In [35]:
layered_losses_tangent_0[1][2]

200-element Array{Tuple{Float64,Float64},1}:
 (69.2365, 69.2365)
 (51.4347, 51.4347)
 (38.8203, 38.8203)
 (34.6861, 34.6861)
 (31.8719, 31.8719)
 (29.9075, 29.9075)
 (28.6035, 28.6035)
 (27.9436, 27.9436)
 (27.5948, 27.5948)
 (27.3777, 27.3777)
 (27.2521, 27.2521)
 (27.1708, 27.1708)
 (27.111, 27.111)  
 ⋮                 
 (21.2752, 21.2752)
 (21.291, 21.291)  
 (21.2961, 21.2961)
 (21.2972, 21.2972)
 (21.3027, 21.3027)
 (21.3099, 21.3099)
 (21.3188, 21.3188)
 (21.33, 21.33)    
 (21.3442, 21.3442)
 (21.3607, 21.3607)
 (21.3788, 21.3788)
 (21.3979, 21.3979)